## total precipitation contribution from rainfall categories

In [ ]:
import sys
import os
import xarray as xr
from glob import glob
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import dask.array as da
from scipy.ndimage import gaussian_filter
from scipy import interpolate
from scipy.stats import linregress

from datetime import datetime, timedelta
import cartopy.crs as ccrs
import cartopy.feature as cfeat
from cartopy.util import add_cyclic_point
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from scipy.ndimage.filters import uniform_filter1d

import warnings
import psutil
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
from matplotlib import patches
from matplotlib import cm
from matplotlib.colors import ListedColormap, Normalize

from metpy import calc as mpcalc
from metpy.units import units
from scipy.stats import ttest_ind

In [ ]:
def running_mean(y,window_N):
    y_avg = np.zeros(len(y))
    avg_mask = np.ones(window_N) / window_N
    y_avg = np.convolve(y, avg_mask, 'same')
    return y_avg

In [ ]:
ISCCP_dir = '/data/willytsai/ISCCP/hgg/'
TRMM_dir = '/data2/willytsai/TRMM_3hr/TRMM/'
MERRA2_dir = '/data2/willytsai/MERRA2/'
MERRA2_3d_dir = '/data/willytsai/'
ERA5_dir = '/data/willytsai/ERA5/'
Gridsat_dir = '/data/willytsai/gridsat_BT/remap_0.1deg/'
fig_dir = '/data/willytsai/ISCCP/fig/'

In [ ]:
# load data
os.chdir('/data/willytsai/ISCCP/')

# variable namelist
var_name = np.loadtxt('var_name_2014_2018_5deg_4ds.dat',dtype='U16')
# event time
var_time = np.loadtxt('time_event_2014_2018_5deg_4ds.dat',dtype='U13')
time_event = []
for t in var_time:
    tmp = datetime.strptime(t,'%Y-%m-%d-%H')
    time_event.append(tmp)

var_dataset = (np.loadtxt('var_event_2014_2018_5deg_4ds.dat'))

In [ ]:
lat_event = var_dataset[:,0]
lon_event = var_dataset[:,1]

In [ ]:
%%time
date = []; date_begin = datetime(2014,1,1)
for t in range(14608):
    tmp = date_begin + t*timedelta(hours=3)
    date.append(tmp)
                
# # TRMM3B42 dataset 2014, 3hrly separate
prec_3hr = np.nan*np.zeros((len(date),241,1440))

os.chdir(TRMM_dir+'2009_2018')
files = sorted(glob('*2014*')+glob('*2015*')+glob('*2016*')+glob('*2017*')+glob('*2018*'))
for n,file in enumerate(files):
    data = xr.open_dataset(file).precipitation        
    prec_3hr[n,:,:] = data.values.swapaxes(0,1)
         
lon = data.nlon; lat = data.nlat
prec_xr = xr.DataArray(prec_3hr,coords=[date,lat,lon],dims=['time','lat','lon']); del prec_3hr

In [ ]:
MERRA2_dir

In [ ]:
# # MERRA-2 CWV dataset 2014, 3hrly separate
cwv_3hr = np.nan*np.zeros((len(date),121,576))

os.chdir(MERRA2_dir+'inst_2d_hourly')
files = sorted(glob('*2014*')+glob('*2015*')+glob('*2016*')+glob('*2017*')+glob('*2018*'))
for n,file in enumerate(files):
    data = xr.open_dataset(file).TQV
    cwv_3hr[8*n:8*(n+1),:,:] = data.resample(time='3H').nearest()
         
lon = data.lon; lat = data.lat
cwv_xr = xr.DataArray(cwv_3hr,coords=[date,lat,lon],dims=['time','lat','lon']); del cwv_3hr

In [ ]:
os.chdir('/data/willytsai/ISCCP/')
prec_all = np.load('prec_all_5deg_4ds.mat.npy') # event

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# testing bed for the Indian Ocean
lon_cen = 70; lat_cen = 5
idx_lon = np.where(lon_event==70)[0]
idx_lat = np.where(lat_event==5)[0]
idx_com = np.intersect1d(idx_lon,idx_lat)

prec_loc = prec_all[idx_com,:]

for n in range(len(prec_bin)-1):
    idx = np.where(np.logical_and(24*prec_loc[:,16]>=prec_bin[n],
                                  24*prec_loc[:,16]<prec_bin[n+1]))[0]

In [ ]:
prec_mn = np.nanmean(prec_xr.sel(lat=slice(lat_cen-2.5,lat_cen+2.5)
                     ,lon=slice(lon_cen-2.5,lon_cen+2.5)),axis=(1,2))
cwv_mn = np.nanmean(cwv_xr.sel(lat=slice(lat_cen-2.5,lat_cen+2.5)
                     ,lon=slice(lon_cen-2.5,lon_cen+2.5)),axis=(1,2))

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,4))
ax.plot(date,24*prec_mn)
ax1 = ax.twinx()
ax1.plot(date,running_mean(cwv_mn,40),'m')
ax1.set_ylim([30,80])

for i in idx:
    plt.plot(time_event[idx_com[i]],20,'^g')
    plt.vlines(x=time_event[idx_com[i]]-timedelta(days=2),ymin=0,ymax=120,color='lightgrey')
    plt.vlines(x=time_event[idx_com[i]]+timedelta(days=2),ymin=0,ymax=120,color='lightgrey')

plt.xlim([datetime(2015,6,1),datetime(2015,8,15)])

In [ ]:
%%time

lat_bin = np.linspace(-7.5,7.5,7)
lon_bin = np.linspace(-177.5,177.5,143)
prec_bin = np.array([5,15,25,35,45,200])

precsum_frac = np.zeros((len(prec_bin)-1,len(lon_bin),len(lat_bin)))
precnum_frac = np.copy(precsum_frac)

for i,lon_cen in enumerate(lon_bin):
    for j,lat_cen in enumerate(lat_bin):
        
        prec_total = np.nansum(np.nanmean(prec_xr.sel(lat=slice(lat_cen-2.5,lat_cen+2.5)
                     ,lon=slice(lon_cen-2.5,lon_cen+2.5)),axis=(1,2))) # time-series
        
        idx_lon = np.where(lon_event==lon_cen)[0]
        idx_lat = np.where(lat_event==lat_cen)[0]
        idx_com = np.intersect1d(idx_lon,idx_lat)
        
        prec_loc = prec_all[idx_com,:]
        
        for n in range(len(prec_bin)-1):
            idx = np.where(np.logical_and(24*prec_loc[:,16]>=prec_bin[n],
                                          24*prec_loc[:,16]<prec_bin[n+1]))[0]
            
            prec_total_event = np.nansum(prec_loc[idx,:])
            precsum_frac[n,i,j] = prec_total_event/prec_total
            try:
                precnum_frac[n,i,j] = len(idx)/len(idx_com)
            except:
                precnum_frac[n,i,j] = 0

In [ ]:
lat_edge = np.linspace(-8.75,8.75,8)
lon_edge = np.linspace(-178.75,181.25,145)

ratio_map_all = np.zeros((5,145,8))
num_map_all = np.copy(ratio_map_all)

for n in range(5):
    ratio_map = np.zeros((145,7))
    ratio_map[:143,:] = precsum_frac[n,:,:]
    ratio_map[143,:] = (precsum_frac[n,0,:]+precsum_frac[n,-1,:])/2
    ratio_map[144,:] = (precsum_frac[n,0,:]+precsum_frac[n,-1,:])/2    
    ratio_map = np.vstack([ratio_map.T,np.zeros((ratio_map.shape[0])).T]).T
    ratio_map[ratio_map==0] = np.nan
    
    num_map = np.zeros((145,7))
    num_map[:143,:] = precnum_frac[n,:,:]
    num_map[143,:] = (precnum_frac[n,0,:]+precnum_frac[n,-1,:])/2
    num_map[144,:] = (precnum_frac[n,0,:]+precnum_frac[n,-1,:])/2    
    num_map = np.vstack([num_map.T,np.zeros((num_map.shape[0])).T]).T
    num_map[num_map==0] = np.nan
    
    ratio_map_all[n,:,:] = ratio_map
    num_map_all[n,:,:] = num_map

In [ ]:
fig = plt.figure(figsize=(13,4))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))

ps = plt.pcolor(lon_edge,lat_edge,ratio_map_all[0,:,:].T,transform=ccrs.PlateCarree(),
                  cmap='jet',vmin=0,vmax=0.3)
cbar = plt.colorbar(fraction=0.05,pad=0.1,orientation='horizontal')
cbar.set_label('fraction of total rainfall',fontsize=12)
#cs = plt.contour(prec_clim.lon,prec_clim.lat,prec_clim.mean(axis=0)*24)
ax.coastlines()
ax.add_feature(cfeat.LAND,zorder=1,edgecolor='k',facecolor='whitesmoke') # maskout LAND

ax.set_yticks([-20,-15,-10,-5,0,5,10,15,20])
ax.set_yticklabels(['20S','15S','10S','5S','EQ','5N','10N','15N','20N'])
ax.set_xticks([-179.875,-120,-60,-0,60,120,179.875])
ax.set_xticklabels(['0','60E','120E','180','120W','60W','0'])
ax.set_ylim([-10,10]); ax.set_xlim([-129,89]);ax.grid(linestyle=':')
ax.set_title('(a) Rainfall contribution, 5-15 (mm/d), 2014-2018',fontsize=15);

#plt.savefig(fig_dir+'rainfrac_p515_4ds.pdf',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(13,4))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))

ps = plt.pcolor(lon_edge,lat_edge,ratio_map_all[1,:,:].T,transform=ccrs.PlateCarree(),
                  cmap='jet',vmin=0,vmax=0.3)
cbar = plt.colorbar(fraction=0.05,pad=0.1,orientation='horizontal')
cbar.set_label('fraction of total rainfall',fontsize=12)
#cs = plt.contour(prec_clim.lon,prec_clim.lat,prec_clim.mean(axis=0)*24)
ax.coastlines()
ax.add_feature(cfeat.LAND,zorder=1,edgecolor='k',facecolor='whitesmoke') # maskout LAND

ax.set_yticks([-20,-15,-10,-5,0,5,10,15,20])
ax.set_yticklabels(['20S','15S','10S','5S','EQ','5N','10N','15N','20N'])
ax.set_xticks([-179.875,-120,-60,-0,60,120,179.875])
ax.set_xticklabels(['0','60E','120E','180','120W','60W','0'])
ax.set_ylim([-10,10]); ax.set_xlim([-129,89]);ax.grid(linestyle=':')
ax.set_title('(b) Rainfall contribution, 15-25 (mm/d), 2014-2018',fontsize=15);

#plt.savefig(fig_dir+'rainfrac_p1525_4ds.pdf',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(13,4))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))

ps = plt.pcolor(lon_edge,lat_edge,ratio_map_all[2,:,:].T,transform=ccrs.PlateCarree(),
                  cmap='jet',vmin=0,vmax=0.3)
cbar = plt.colorbar(fraction=0.05,pad=0.1,orientation='horizontal')
cbar.set_label('fraction of total rainfall',fontsize=12)
#cs = plt.contour(prec_clim.lon,prec_clim.lat,prec_clim.mean(axis=0)*24)
ax.coastlines()
ax.add_feature(cfeat.LAND,zorder=1,edgecolor='k',facecolor='whitesmoke') # maskout LAND

ax.set_yticks([-20,-15,-10,-5,0,5,10,15,20])
ax.set_yticklabels(['20S','15S','10S','5S','EQ','5N','10N','15N','20N'])
ax.set_xticks([-179.875,-120,-60,-0,60,120,179.875])
ax.set_xticklabels(['0','60E','120E','180','120W','60W','0'])
ax.set_ylim([-10,10]); ax.set_xlim([-129,89]);ax.grid(linestyle=':')
ax.set_title('(c) Rainfall contribution, 25-35 (mm/d), 2014-2018',fontsize=15);

#plt.savefig(fig_dir+'rainfrac_p2535_4ds.pdf',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(13,4))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))

ps = plt.pcolor(lon_edge,lat_edge,ratio_map_all[3,:,:].T,transform=ccrs.PlateCarree(),
                  cmap='jet',vmin=0,vmax=0.3)
cbar = plt.colorbar(fraction=0.05,pad=0.1,orientation='horizontal')
cbar.set_label('fraction of total rainfall',fontsize=12)
#cs = plt.contour(prec_clim.lon,prec_clim.lat,prec_clim.mean(axis=0)*24)
ax.coastlines()
ax.add_feature(cfeat.LAND,zorder=1,edgecolor='k',facecolor='whitesmoke') # maskout LAND

ax.set_yticks([-20,-15,-10,-5,0,5,10,15,20])
ax.set_yticklabels(['20S','15S','10S','5S','EQ','5N','10N','15N','20N'])
ax.set_xticks([-179.875,-120,-60,-0,60,120,179.875])
ax.set_xticklabels(['0','60E','120E','180','120W','60W','0'])
ax.set_ylim([-10,10]); ax.set_xlim([-129,89]);ax.grid(linestyle=':')
ax.set_title('(d) Rainfall contribution, 35-45 (mm/d), 2014-2018',fontsize=15);

#plt.savefig(fig_dir+'rainfrac_p3545_4ds.pdf',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(13,4))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))

ps = plt.pcolor(lon_edge,lat_edge,ratio_map_all[4,:,:].T,transform=ccrs.PlateCarree(),
                  cmap='jet',vmin=0)
cbar = plt.colorbar(fraction=0.05,pad=0.1,orientation='horizontal')
cbar.set_label('fraction of total rainfall',fontsize=12)
#cs = plt.contour(prec_clim.lon,prec_clim.lat,prec_clim.mean(axis=0)*24)
ax.coastlines()
ax.add_feature(cfeat.LAND,zorder=1,edgecolor='k',facecolor='whitesmoke') # maskout LAND

ax.set_yticks([-20,-15,-10,-5,0,5,10,15,20])
ax.set_yticklabels(['20S','15S','10S','5S','EQ','5N','10N','15N','20N'])
ax.set_xticks([-179.875,-120,-60,-0,60,120,179.875])
ax.set_xticklabels(['0','60E','120E','180','120W','60W','0'])
ax.set_ylim([-10,10]); ax.set_xlim([-129,89]);ax.grid(linestyle=':')
ax.set_title('(e) Rainfall contribution, 45 up (mm/d), 2014-2018',fontsize=15);

#plt.savefig(fig_dir+'rainfrac_p45up_4ds.pdf',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(13,4))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))

ps = plt.pcolor(lon_edge,lat_edge,num_map_all[0,:,:].T,transform=ccrs.PlateCarree(),
                  cmap='jet')
cbar = plt.colorbar(fraction=0.05,pad=0.1,orientation='horizontal')
cbar.set_label('fraction of total events',fontsize=12)
#cs = plt.contour(prec_clim.lon,prec_clim.lat,prec_clim.mean(axis=0)*24)
ax.coastlines()
ax.add_feature(cfeat.LAND,zorder=1,edgecolor='k',facecolor='whitesmoke') # maskout LAND

ax.set_yticks([-20,-15,-10,-5,0,5,10,15,20])
ax.set_yticklabels(['20S','15S','10S','5S','EQ','5N','10N','15N','20N'])
ax.set_xticks([-179.875,-120,-60,-0,60,120,179.875])
ax.set_xticklabels(['0','60E','120E','180','120W','60W','0'])
ax.set_ylim([-10,10]); ax.set_xlim([-129,89]);ax.grid(linestyle=':')
ax.set_title('(a) event fraction, 5-15 (mm/d), 2014-2018',fontsize=15);

#plt.savefig(fig_dir+'rainfrac_p45up_4ds.pdf',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(13,4))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))

ps = plt.pcolor(lon_edge,lat_edge,num_map_all[1,:,:].T,transform=ccrs.PlateCarree(),
                  cmap='jet',vmin=0,vmax=0.3)
cbar = plt.colorbar(fraction=0.05,pad=0.1,orientation='horizontal')
cbar.set_label('fraction of total events',fontsize=12)
#cs = plt.contour(prec_clim.lon,prec_clim.lat,prec_clim.mean(axis=0)*24)
ax.coastlines()
ax.add_feature(cfeat.LAND,zorder=1,edgecolor='k',facecolor='whitesmoke') # maskout LAND

ax.set_yticks([-20,-15,-10,-5,0,5,10,15,20])
ax.set_yticklabels(['20S','15S','10S','5S','EQ','5N','10N','15N','20N'])
ax.set_xticks([-179.875,-120,-60,-0,60,120,179.875])
ax.set_xticklabels(['0','60E','120E','180','120W','60W','0'])
ax.set_ylim([-10,10]); ax.set_xlim([-129,89]);ax.grid(linestyle=':')
ax.set_title('(b) event fraction, 15-25 (mm/d), 2014-2018',fontsize=15);

#plt.savefig(fig_dir+'rainfrac_p45up_4ds.pdf',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(13,4))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))

ps = plt.pcolor(lon_edge,lat_edge,num_map_all[2,:,:].T,transform=ccrs.PlateCarree(),
                  cmap='jet',vmin=0,vmax=0.3)
cbar = plt.colorbar(fraction=0.05,pad=0.1,orientation='horizontal')
cbar.set_label('fraction of total events',fontsize=12)
#cs = plt.contour(prec_clim.lon,prec_clim.lat,prec_clim.mean(axis=0)*24)
ax.coastlines()
ax.add_feature(cfeat.LAND,zorder=1,edgecolor='k',facecolor='whitesmoke') # maskout LAND

ax.set_yticks([-20,-15,-10,-5,0,5,10,15,20])
ax.set_yticklabels(['20S','15S','10S','5S','EQ','5N','10N','15N','20N'])
ax.set_xticks([-179.875,-120,-60,-0,60,120,179.875])
ax.set_xticklabels(['0','60E','120E','180','120W','60W','0'])
ax.set_ylim([-10,10]); ax.set_xlim([-129,89]);ax.grid(linestyle=':')
ax.set_title('(c) event fraction, 25-35 (mm/d), 2014-2018',fontsize=15);

#plt.savefig(fig_dir+'rainfrac_p45up_4ds.pdf',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(13,4))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))

ps = plt.pcolor(lon_edge,lat_edge,num_map_all[3,:,:].T,transform=ccrs.PlateCarree(),
                  cmap='jet',vmin=0,vmax=0.3)
cbar = plt.colorbar(fraction=0.05,pad=0.1,orientation='horizontal')
cbar.set_label('fraction of total events',fontsize=12)
#cs = plt.contour(prec_clim.lon,prec_clim.lat,prec_clim.mean(axis=0)*24)
ax.coastlines()
ax.add_feature(cfeat.LAND,zorder=1,edgecolor='k',facecolor='whitesmoke') # maskout LAND

ax.set_yticks([-20,-15,-10,-5,0,5,10,15,20])
ax.set_yticklabels(['20S','15S','10S','5S','EQ','5N','10N','15N','20N'])
ax.set_xticks([-179.875,-120,-60,-0,60,120,179.875])
ax.set_xticklabels(['0','60E','120E','180','120W','60W','0'])
ax.set_ylim([-10,10]); ax.set_xlim([-129,89]);ax.grid(linestyle=':')
ax.set_title('(d) event fraction, 35-45 (mm/d), 2014-2018',fontsize=15);

#plt.savefig(fig_dir+'rainfrac_p45up_4ds.pdf',bbox_inches='tight')

In [ ]:
fig = plt.figure(figsize=(13,4))
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))

ps = plt.pcolor(lon_edge,lat_edge,num_map_all[3,:,:].T,transform=ccrs.PlateCarree(),
                  cmap='jet',vmin=0,vmax=0.3)
cbar = plt.colorbar(fraction=0.05,pad=0.1,orientation='horizontal')
cbar.set_label('fraction of total rainfall',fontsize=12)
#cs = plt.contour(prec_clim.lon,prec_clim.lat,prec_clim.mean(axis=0)*24)
ax.coastlines()
ax.add_feature(cfeat.LAND,zorder=1,edgecolor='k',facecolor='whitesmoke') # maskout LAND

ax.set_yticks([-20,-15,-10,-5,0,5,10,15,20])
ax.set_yticklabels(['20S','15S','10S','5S','EQ','5N','10N','15N','20N'])
ax.set_xticks([-179.875,-120,-60,-0,60,120,179.875])
ax.set_xticklabels(['0','60E','120E','180','120W','60W','0'])
ax.set_ylim([-10,10]); ax.set_xlim([-129,89]);ax.grid(linestyle=':')
ax.set_title('(d) event fraction, 35-45 (mm/d), 2014-2018',fontsize=15);

#plt.savefig(fig_dir+'rainfrac_p45up_4ds.pdf',bbox_inches='tight')